In [ ]:
# Uninstall tf-nightly if installed
!pip uninstall -y tf-nightly keras-nightly tb-nightly tensorflow-estimator ml-dtypes

# Install stable TensorFlow version and datasets
!pip install -U tensorflow==2.15.0
!pip install -U tensorflow-datasets

# Import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D, Dense, Dropout

# Load data from .tsv files
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

# Normalize labels
train_data['label'] = train_data['label'].str.strip().str.lower()
test_data['label'] = test_data['label'].str.strip().str.lower()

# Map labels to 0 and 1
label_map = {'ham': 0, 'spam': 1}
train_data['label'] = train_data['label'].map(label_map)
test_data['label'] = test_data['label'].map(label_map)

# Text Vectorization
max_features = 5000
sequence_length = 120

vectorizer = TextVectorization(max_tokens=max_features, output_mode='int', output_sequence_length=sequence_length)
vectorizer.adapt(train_data['text'].values)

# Build the model
model = Sequential([
    vectorizer,
    Embedding(input_dim=max_features + 1, output_dim=16),
    GlobalAveragePooling1D(),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prepare data for training
train_ds = tf.data.Dataset.from_tensor_slices((train_data['text'].values, train_data['label'].values)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_data['text'].values, test_data['label'].values)).batch(32)

# Train the model
model.fit(train_ds, validation_data=test_ds, epochs=30)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prob = model.predict([pred_text])[0][0]
    label = 'spam' if prob > 0.5 else 'ham'
    print(f"Message: \"{pred_text}\"\n → Prediction: {prob:.4f} → {label}\n")
    return [prob, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
